In [24]:
import numpy as np
import random
import copy
from itertools import product
from scipy import stats 
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### Helper Functions

In [25]:
# modified combinations with replacement function so results include those which are not sorted 
def combinations_with_replacement(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in product(range(n), repeat=r):
        yield tuple(pool[i] for i in indices)

In [26]:
# returns a combined list with no repeats 
def combined_list(listA, listB):
    set1 = set(listA)
    set2 = set(listB)
    
    combined = listA + list(set2 - set1)
    return combined

In [27]:
# returns all possible appointment schedules given N patients and F slots, where each slot is assigned at least one patient      
def all_schedules(N, F):
    A1 = []
    slots = (range(1, F+1))
    possibilities = list(combinations_with_replacement(slots, N))
    for p in possibilities: 
        p = list(p)
        if (all (x in p for x in list(slots))):
            A1.append(p)
    return A1

In [28]:
# transform schedule into dictionary
def transform_schedule(schedule):
    schedule_dict = {}
    for i in range(len(schedule)):
        if schedule[i] not in schedule_dict:
            schedule_dict[schedule[i]] = list()
        schedule_dict[schedule[i]].append(i)
    return (schedule_dict)

In [29]:
# load dataset from previous program 
sample_dataset = np.load('simulation_dataset.npy')

# helper function that will randomly select N patients from the dataset 
# returns a list of N show probabilities, from least to greatest 
def patient_selection(N):
    indices = np.random.choice(len(sample_dataset), N, replace = True)
    selected = sample_dataset[indices]
    
    # sort everything by the show probabilities in increasing order
    sorted_list = selected[selected[:, 2].argsort()]
    return (sorted_list)

### Strategy Simulation Functions

In [30]:
def TOF0(schedule, N, F, iters):
    avg_show_rate = 1 - 0.2018964550044335
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    male_p_wait_list = []
    female_p_wait_list = []
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(avg_show_rate)        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        if len(G_m) > 0:
            avg = np.nanmean(np.array(p_wait[G_m]))
            if(np.isnan(avg) == False):
                male_p_wait_list.append(avg)    
        if len(G_f) > 0:
            avg = np.nanmean(np.array(p_wait[G_f]))
            if(np.isnan(avg) == False):
                female_p_wait_list.append(avg) 
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
    return it_list, ot_list, p_wait_list, male_p_wait_list, female_p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list

In [31]:
def TOF2(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []
    # for the demographic groups - waiting times
    male_p_wait_list = []
    female_p_wait_list = []
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        if len(G_m) > 0:
            avg = np.nanmean(np.array(p_wait[G_m]))
            if(np.isnan(avg) == False):
                male_p_wait_list.append(avg)    
        if len(G_f) > 0:
            avg = np.nanmean(np.array(p_wait[G_f]))
            if(np.isnan(avg) == False):
                female_p_wait_list.append(avg) 
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
    return it_list, ot_list, p_wait_list, male_p_wait_list, female_p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list

In [32]:
def UOF(schedule, N, F, iters):
    # key metrics 
    it_list = []
    ot_list = []
    p_wait_list = []

    # for the demographic groups - waiting times
    male_p_wait_list = []
    female_p_wait_list = []
    scholarship_p_wait_list = []
    noscholarship_p_wait_list = []
    
    # additional metrics for UOF
    expected_shows_list = []
    expected_shows_m_list = []
    expected_shows_f_list = []
    expected_shows_s_list = []
    expected_shows_ns_list = []
        
    planned = transform_schedule(schedule)

    for i in range(iters):
        # randomly select patients N from master list and their show probabilities
        # sort them in order from least show to greatest show probability 
        patient_sample = patient_selection(N)
        # divide patients into groups 
        G_m = []
        G_f = []
        G_s = []
        G_ns = []
        show_prob = []
        for i in range(N):
            if(patient_sample[i][0] == 1):
                G_m.append(i)
            else:
                G_f.append(i)
            if(patient_sample[i][1] == 1):
                G_s.append(i)
            else:
                G_ns.append(i)
            show_prob.append(patient_sample[i][2])        
        
        # generate random numbers for all N patients to determine whether or not they show up 
        # show = True if patient shows up, False if patient is no-show 
        U = np.random.rand(len(show_prob))
        show = U < show_prob
        
        # variables for what we are keeping track of
        # array of N patient wait times 
        p_wait = np.zeros(len(show_prob))
        # total provider idle time
        it = 0
        # total provider over time
        ot = 0
        
        patients_left = N
        temp = []
        actual = copy.deepcopy(planned)

        # iterate through all slots in the planned day 
        for slot in range(1, F+1):
            patients = (actual[slot])
            shows = []
            noshows = []
            
            if (isinstance(patients, int)):
                if(show[patients]):
                    shows.append(patients)
                else:
                    noshows.append(patients)
            # for each patient assigned to the slot
            else:
                for p in patients:
                    if(show[p] == True):
                        shows.append(p)
                    else:
                        noshows.append(p)
                        p_wait[p] = np.nan
                        
            patients_left -= len(noshows)        

            # no patient arrivals here: doctores are idle       
            if len(shows) == 0:
                it += 1

            # more than 1 patient arrival for slot 1
            if len(shows) > 1:
                # pick a patient at random to see first 
                if not temp:
                    lucky_patient = random.choice(shows)
                else:
                    # out of all patients in waiting room, pick from the ones who have been waiting longest 
                    temp_waiting = [p_wait[p] for p in temp]
                    #print("temp waiting", temp_waiting)
                    longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())

                    # if there is only one patient that is waiting the longest, that one is seen first
                    if(isinstance(longest_wait, int)):
                        lucky_patient = temp[longest_wait]
                    # if there is a tie
                    else:
                        longest_wait_patient = [temp[p] for p in longest_wait]
                        lucky_patient = random.choice(longest_wait_patient)

                    temp.remove(lucky_patient)

                # other patients get moved to next time 
                unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
                # if only one unlucky patient
                if(isinstance(unlucky_patients, int)):
                    if(unlucky_patients not in temp):
                        temp.extend(unlucky_patients)
                else:
                    temp = combined_list(temp, unlucky_patients)

                for x in unlucky_patients:
                    # remove them from their origial time slot 
                    actual[slot].remove(x)
                    # move them to the next slot (slot + 1)
                    if (slot+1) not in actual:
                        actual[slot+1] = list()
                    actual[slot+1].append(x)
                    # increase the wait time for unlucky patients
                    p_wait[x] += 1

            # handle the no shows 
            actual[slot] = np.setdiff1d(actual[slot], noshows).tolist()

            # decrease counter
            if(len(shows) >= 1):
                patients_left -= 1

        # overtime simulation 
        while(patients_left > 0):
            ot += 1
            # examine patients in slot F + ot 
            current_slot = F + ot
            # shows are all patients in the current slot 
            shows = actual[current_slot]
            # only need to consider patients in waiting room 
            temp_waiting = [p_wait[p] for p in temp]
            longest_wait = (np.squeeze(np.argwhere(temp_waiting == np.amax(temp_waiting))).tolist())
            if(isinstance(longest_wait, int)):
                lucky_patient = temp[longest_wait]   
            else:
                longest_wait_patient = [temp[p] for p in longest_wait]
                lucky_patient = random.choice(longest_wait_patient)

            temp.remove(lucky_patient)

            unlucky_patients = np.setdiff1d(shows, lucky_patient).tolist()
            temp.extend(unlucky_patients) 
            for x in unlucky_patients:
                # remove them from their origial time slot 
                actual[current_slot].remove(x)
                # move them to the next slot (slot + 1)
                if (current_slot+1) not in actual:
                    actual[current_slot+1] = list()
                actual[current_slot+1].append(x)
                # increase the wait time for unlucky patients
                p_wait[x] += 1

            patients_left -= 1

        it_list.append(it)
        ot_list.append(ot)
        p_wait_list.append(np.nansum(p_wait))
        
        # Number of shows is equal to the number of patients whose wait time is not nan
        expected_shows_list.append(~np.sum(np.isnan(p_wait)))
        
        if len(G_m) > 0:
            avg = np.nanmean(np.array(p_wait[G_m]))
            if(np.isnan(avg) == False):
                male_p_wait_list.append(avg) 
        if len(G_f) > 0:
            avg = np.nanmean(np.array(p_wait[G_f]))
            if(np.isnan(avg) == False):
                female_p_wait_list.append(avg) 
        if len(G_s) > 0:
            avg = np.nanmean(np.array(p_wait[G_s]))
            if(np.isnan(avg) == False):
                scholarship_p_wait_list.append(avg) 
        if len(G_ns) > 0:
            avg = np.nanmean(np.array(p_wait[G_ns]))
            if(np.isnan(avg) == False):
                noscholarship_p_wait_list.append(avg) 
        
        # flag for determining the group with max wait time (USING SCHOLARSHIP FOR NOW)
        scholarship_wait_isLonger = 0
        if (np.mean(scholarship_p_wait_list) > np.mean(noscholarship_p_wait_list)):
            scholarship_wait_isLonger == 1
        else:
            scholarship_wait_isLonger == 0
    return it_list, ot_list, p_wait_list, male_p_wait_list, female_p_wait_list, scholarship_p_wait_list, noscholarship_p_wait_list, expected_shows_list, scholarship_wait_isLonger 

### Simulation to Find Optimal Schedule

In [54]:
def find_optimal(strategy, N, F, rho, tao, omega, iters):
    
    schedule_performance = []
    schedule_performance_std = []
    possible_schedules = all_schedules(N, F)
    shows = []
    s_wait_isLonger = []
    ns_wait_isLonger = []
    
    for s in possible_schedules: 
        if(strategy == "TOF2"):
            it, ot, wt, male_wt, female_wt, scholarship_wt, noscholarship_wt = (TOF2(s, N, F, iters))
        if(strategy == "TOF0"):
            it, ot, wt, male_wt, female_wt, scholarship_wt, noscholarship_wt = (TOF0(s, N, F, iters))
            
        if(strategy == "UOF"):
            it, ot, wt, male_wt, female_wt, scholarship_wt, noscholarship_wt, E_shows, s_isLonger = (UOF(s, N, F, iters))
            shows.append(np.mean(E_shows))
            s_wait_isLonger.append(s_isLonger)
            ns_wait_isLonger.append(1 - s_isLonger)
            
        schedule_performance.append([np.mean(it), np.mean(ot), np.mean(wt), np.mean(male_wt), np.mean(female_wt), np.mean(scholarship_wt), np.mean(noscholarship_wt)])
        schedule_performance_std.append([np.std(it), np.std(ot), np.std(wt), np.std(male_wt), np.std(female_wt), np.std(scholarship_wt), np.std(noscholarship_wt)])
        
    # calcuate optimal based on objective function 
    if(strategy == "TOF2" or strategy == "TOF0"):
        objective = []
        objective = [rho*schedule_performance[x][0] + tao*schedule_performance[x][1] + omega*schedule_performance[x][2] for x in range(len(schedule_performance))]

    elif(strategy == "UOF"):
        print(len(shows))
        print(len(schedule_performance))
        W_max = []
        for i in range(len(possible_schedules)):
            #print(i)
            if (s_wait_isLonger[i] == 1):
                W_max.append(shows[i] * schedule_performance[i][5])
            else:
                W_max.append(shows[i] * schedule_performance[i][6])
             
        objective = []
        objective = [rho*schedule_performance[x][0] + tao*schedule_performance[x][1] + omega*W_max[x] for x in range(len(schedule_performance))]
    
    opt = np.argmin(objective)
    print(possible_schedules[opt])
    print(transform_schedule(possible_schedules[opt]))
    print(schedule_performance[opt])
    
    # test for statistical significance
    if(strategy == "TOF2"):
        it, ot, wt, male_wt, female_wt, scholarship_wt, noscholarship_wt = (TOF2(possible_schedules[opt], N, F, iters))
    if(strategy == "TOF0"):
        it, ot, wt, male_wt, female_wt, scholarship_wt, noscholarship_wt = (TOF0(possible_schedules[opt], N, F, iters))
    if(strategy == "UOF"):
        it, ot, wt, male_wt, female_wt, scholarship_wt, noscholarship_wt, E_show, scholarship_isLonger = (UOF(possible_schedules[opt], N, F, iters))
    # Fairness of gender
    print("Gender Fairness")
    print("Male average wait time: ", np.mean(male_wt))
    print("Female average wait time: ", np.mean(female_wt))
    print("Statistical significance test: ")
    print(stats.ttest_ind(male_wt, female_wt))
    print("Scholarship Fairness")
    # Fairness of scholarship 
    print("Scholarship average wait time: ", np.mean(scholarship_wt))
    print("No scholarship average wait time: ", np.mean(noscholarship_wt))
    print("Statistical significance test: ")
    print(stats.ttest_ind(scholarship_wt, noscholarship_wt))
    
    return (possible_schedules[opt], schedule_performance[opt])

In [56]:
np.random.seed(0)

iters = 50
# number of appointment requests
N = 6
# number of appointment slots 
F = 4

# idle time cost 
rho = 1
# overtime cost 
tao = 1.5
# waiting time cost
omega = 0.5


print("UOF")
y1, y2 = find_optimal("UOF", N, F, rho, tao, omega, iters)
print()
print("TOF2")
x1, x2 = find_optimal("TOF2", N, F, rho, tao, omega, iters)
print()

print()
print("TOF0")
z1, z2 = find_optimal("TOF0", N, F, rho, tao, omega, iters)

UOF
1560
1560
[4, 3, 1, 2, 2, 1]
{4: [0], 3: [1], 1: [2, 5], 2: [3, 4]}
[0.0, 1.5, 6.78, 1.2806201550387597, 1.2373333333333334, 1.1904761904761905, 1.1933333333333334]
Gender Fairness
Male average wait time:  1.1768939393939393
Female average wait time:  1.309333333333333
Statistical significance test: 
Ttest_indResult(statistic=-1.6127686792261013, pvalue=0.11022000417282267)
Scholarship Fairness
Scholarship average wait time:  1.4791666666666667
No scholarship average wait time:  1.2536666666666667
Statistical significance test: 
Ttest_indResult(statistic=2.6022506154842313, pvalue=0.011236852493335388)

TOF2
[4, 4, 2, 4, 1, 3]
{4: [0, 1, 3], 2: [2], 1: [4], 3: [5]}
[0.02, 1.68, 2.38, 0.3829545454545454, 0.4256666666666667, 0.3888888888888889, 0.41700000000000004]
Gender Fairness
Male average wait time:  0.3299242424242424
Female average wait time:  0.5296666666666667
Statistical significance test: 
Ttest_indResult(statistic=-2.572013823970765, pvalue=0.011712902661131163)
Scholarsh

In [52]:


iters = 10000
# number of appointment requests
N = 6
# number of appointment slots 
F = 4

# idle time cost 
rho = 1
# overtime cost 
tao = 1.5
# waiting time cost
omega = 0.5

x1, x2 = find_optimal("TOF2", N, F, rho, tao, omega, iters)
print()
y1, y2 = find_optimal("TOF0", N, F, rho, tao, omega, iters)

KeyboardInterrupt: 